<a href="https://colab.research.google.com/github/Nastya880/cuda/blob/main/Lab1Ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №1**

**Основы работы с технологией CUDA. Гибридное программирование. Работа с глобальной памятью**

# **Контрольные вопросы**

1. Что такое гибридное программирование?

```
Гибридное программирование - это написание программы для гетерогенной
аппаратной вычислительной структуры, например, для системы, состоящей
из центрального процессора CPU и графического ускорителя GPU.
```
2. Что такое CUDA?

```
CUDA (Compute Unified Device Architecture) – 
- технология (библиотеки и расширенный Cи), предназначенная для разработки
приложений для массивно-параллельных вычислительных устройств, заметно облегчает
написание GPGPU (General Purposed Graphical Processing Unit)-приложений;
- программно-аппаратная архитектура.
```
3. Основные положения программной модели CUDA?

```
GPU (Graphical Processing Unit, device) – это вычислительное устройство,
которое:
-    состоит из массива потоковых мультипроцессоров (streaming 
     multiprocessor, SM);
-    является сопроцессором к центральному процессору CPU (host);
-    имеет собственную память (DRAM);
-    выполняет одновременно большое количество нитей.
```

4. Из чего состоит программный стек CUDA?

```
CPU (Application -> CUDA Libraries, CUDA RunTime, CUDA Drver) -> GPU
```

5. Что такое ядро в CUDA?

```
Ядро (kernel) – это функция, которая работает на GPU и которая 
может быть вызвана только с CPU. Ядро выполняется на сетке из блоков.
```

6. Какие расширения языка Си вводятся в CUDA?

```
Вводимые в CUDA расширения языка Си состоят из: 
-    спецификаторов функций, показывающих, где будет выполняться функция и откуда она может быть вызвана;  
-    спецификаторов переменных, задающих тип памяти, используемый для данных переменных;
-     директивы для запуска ядра из кода;
-    встроенные переменные, содержащие информацию о текущей нити;
-    дополнительные типы данных.
```

7. Какие встроенные переменные поддерживаются в CUDA и для чего они нужны?

```
В CUDA поддерживаются следующие встроенные переменные,
содержащие информацию о текущей нити (рисунок 3):
-    dim3  gridDim;  // размер сетки
-    uint3 blockIdx; // индекс текущего блока в сетке
-    dim3  blockDim; // размер блока 
-    uint3 threadIdx; // индекс текущей нити в блоке
-    int   warpSize; // размер warp’а
Встроенные переменные доступны в функции ядра.
```

8. Какие ограничения вводятся на функции, выполняемые на GPU?

```
- нельзя брать адрес функции (за исключением __global__);
-    не поддерживается рекурсия;
-    не поддерживаются static-переменные внутри функции;
-    не поддерживается переменное число входных аргументов.
```
9. ДОП Какие ограничения вводятся на спецификаторы переменных, выполняемые на GPU?

```
-    нельзя применять к полям структуры (struct или union);
-    не могут быть extern;
-    запись в __constant__  может выполнять только CPU через специальные функции;
-    __shared__ - переменные не могут инициализироваться при объявлении.
```

Настройка для запуска CUDA

In [ ]:
!git config --global url."https://github.com/".insteadOf git://github.com/
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qtej1alk
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qtej1alk
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=395eb04885eff6128cbc2f1c4435bc8cbc11f23432619800a1020a0bbfa435a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-8kke5f0_/wheels/69/4f/dd/2613eddd1a84d8809a21caa3a45ccaae9eac9279231c2feabc
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
!sudo ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!sudo ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


# CUDA НАСТРОЕНА!

**Задание 1. В MS Visual Studio создать проект CUDA VS Wizard. Ознакомиться и запустить программу «Hello world». Получить информацию об устройстве. Измерить время выполнения программы. Результаты занести в отчёт. Запустить программу «Hello world» на всех мультипроцессорах в GPU. Измерить время выполнения программы. Результаты занести в отчёт.**

1 часть. Запуск программы "Hello world"

In [6]:
%%writefile hello.cu
//функция hello, исполняемая на GPU
//kernelName<<Dg, Db, Ns, S>>(args)
//Dg - переменная dim3, размер сетки в блоках
//Db - переменная dim3, размер блока в нитях
//Ns - переменная size_t, объем разделяемой памяти в байтах для каждого блока
//S - поток для вызова

#include <stdio.h>

__global__ void hello_kernel(void)
{
  printf("Hello, world from the device!\n");
}

int main(void)
{
  printf("Hello, world from the host!\n");

  hello_kernel<<<1,1>>>();

  cudaDeviceSynchronize();
  return 0;
}

Overwriting hello.cu


In [7]:
!nvcc hello.cu -o hello -Wno-deprecated-gpu-targets
!nvprof ./hello

Hello, world from the host!
==375== NVPROF is profiling process 375, command: ./hello
Hello, world from the device!
==375== Profiling application: ./hello
==375== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  89.054us         1  89.054us  89.054us  89.054us  hello_kernel(void)
      API calls:   99.81%  346.99ms         1  346.99ms  346.99ms  346.99ms  cudaLaunchKernel
                    0.11%  371.18us         1  371.18us  371.18us  371.18us  cuDeviceTotalMem
                    0.04%  147.83us       101  1.4630us     134ns  62.720us  cuDeviceGetAttribute
                    0.03%  108.65us         1  108.65us  108.65us  108.65us  cudaDeviceSynchronize
                    0.01%  33.971us         1  33.971us  33.971us  33.971us  cuDeviceGetName
                    0.00%  5.4320us         1  5.4320us  5.4320us  5.4320us  cuDeviceGetPCIBusId
                    0.00%  1.5250us         3     508ns     204ns 

2 часть. Информация об устройстве

Информация о возможностях GPU возвращается в виде структуры cudaDeviceProp
struct cudaDeviceProp
{
    char name[256];           //название устройства
    size_t totalGlobalMem;    //полный объем глобальной памяти в байтах
    size_t sharedMemPerBlock; //объем разделяемой памяти в блоке в байтах
    int regsPerBlock;         //кол-во 32битовых регистров в блоке
    int warpSize;             //размер warp
    size_t memPitch;          //макс pitch в байтах, допустимый функциями копирования
                              //памяти, выделенной через cudaMallocPitch

    int maxThreadsPerBlock;   //макс кол-во активных нитей в блоке
    int maxThreadsDim[3];     //макс размер блока
    int maxGridSize[3];       //макс размер сетки
    size_t totalConstMem;     //объем константной памяти
    int major;                //compute capability, старший номер
    int minor;                 //compute capability, младший номер
    int clockRate;            //частота в килогерцах
    size_t textureAlignment;  //выравнивание памяти для текстур
    int deviceOverlap;        //можно ли осуществлять копирование параллельно с вычислениями

    int multiProcessorCount;  //количетсво мультипроцессоров в GPU
    int kernelExecTimeOutEnables; //1, если есть ограничение на время выполнения ядра

    int integrated;           //1, если GPU встроено в материнскую плату
    int canMapHostMemory;     //1, если можно отбражать памяти CPU в куда для использования cudaHostAlloc, cudaHostGetDevicePointer

    int computeMode;          //режим, в котором находится GPU
                              // cudeComputeModeDefault
                              // cudaComputeModeExclusive - только одна нить, может вызвать cudaSetDevice для данного GPU
                              // cudaComputeModeProhibited - ни одна нить не может вызвать cudaSetDevice для данного GPU
}

In [76]:
%%writefile deviceInfo.cu
#include <cstdio>
#include <cuda_runtime.h>
#include <chrono>

int main (int argc, char * argv [] )
{
    int deviceCount;
    cudaDeviceProp devProp{};
    cudaGetDeviceCount ( &deviceCount );
    printf ( "Found %d devices\n", deviceCount );
    for ( int device = 0; device < deviceCount; device++)
    {cudaGetDeviceProperties ( &devProp, device );
        printf ("Device %d\n", device );
        printf ("Compute capability : %d.%d\n", devProp.major, devProp.minor);
        printf ("Name : %s\n", devProp.name);
        // Полный объем глобальной памяти в Mбайтах:
        printf ("Total Global Mem: %lu\n", (devProp.totalGlobalMem/(1024*1024)));
        printf ("Shared memory per block: %zu\n" , devProp.sharedMemPerBlock );
        printf ("Registers per block : %d\n", devProp.regsPerBlock);
        printf ("Warp size : %d\n", devProp.warpSize);
        printf ("Max threads per block: %d\n", devProp.maxThreadsPerBlock);
        printf ("Total constant memory: %zu\n", devProp.totalConstMem);
        printf ("Clock Rate : %d\n", devProp.clockRate);
        printf ("Texture Alignment : %zu\n", devProp.textureAlignment);
        printf ("Device Overlap : %d\n", devProp.deviceOverlap);
        printf ("Multiprocessor Count: %d\n", devProp.multiProcessorCount);
        printf ("Max Threads Dim : %d %d %d\n", devProp.maxThreadsDim[0],
                devProp.maxThreadsDim[1], devProp.maxThreadsDim[2] );
        printf ("Max Grid Size : %d %d %d\n", devProp.maxGridSize [0],
                devProp.maxGridSize [1], devProp.maxGridSize [2]);
    }
    return 0;
}

Writing deviceInfo.cu


In [77]:
!nvcc deviceInfo.cu -o deviceInfo -Wno-deprecated-gpu-targets
!nvprof ./deviceInfo

==2089== NVPROF is profiling process 2089, command: ./deviceInfo
Found 1 devices
Device 0
Compute capability : 7.5
Name : Tesla T4
Total Global Mem: 15109
Shared memory per block: 49152
Registers per block : 65536
Warp size : 32
Max threads per block: 1024
Total constant memory: 65536
Clock Rate : 1590000
Texture Alignment : 512
Device Overlap : 1
Multiprocessor Count: 40
Max Threads Dim : 1024 1024 64
Max Grid Size : 2147483647 65535 65535
==2089== Profiling application: ./deviceInfo
==2089== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   53.66%  378.44us         1  378.44us  378.44us  378.44us  cuDeviceTotalMem
                   21.11%  148.90us       101  1.4740us     129ns  63.768us  cuDeviceGetAttribute
                   18.50%  130.46us         1  130.46us  130.46us  130.46us  cudaGetDeviceProperties
                    5.42%  38.207us         1  38.207us  38.207us  38.207us  cuDev

In [78]:
%%writefile helloTime.cu

#include <stdio.h>
#include <chrono>

__global__ void hello_kernel(void)
{
  printf("Hello, world from the device!\n");
}

int main(void)
{
  /*unsigned int timer;
  cutCreateTimer(&timer);
  curStartTimer(timer);*/

  printf("Hello, world from the host!\n");

  // инициализируем события
  cudaEvent_t start, stop;
  float elapsedTime;

  // создаем события
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  // запись события
  cudaEventRecord(start, 0);
  cudaEventRecord(stop,0);

  hello_kernel<<<1,1>>>(); 
 
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&elapsedTime, start, stop);

  // вывод информации 
  printf("Time spent executing by the GPU: %.7f  millseconds\n", elapsedTime);

  // уничтожение события 
  cudaEventDestroy(start);
  cudaEventDestroy(stop);

  return 0;
}

Writing helloTime.cu


In [79]:
!nvcc helloTime.cu -o helloTime -Wno-deprecated-gpu-targets
!nvprof ./helloTime

Hello, world from the host!
==2140== NVPROF is profiling process 2140, command: ./helloTime
Time spent executing by the GPU: 0.0038080  millseconds
==2140== Profiling application: ./helloTime
==2140== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  88.926us         1  88.926us  88.926us  88.926us  hello_kernel(void)
      API calls:   99.79%  305.22ms         2  152.61ms  1.0210us  305.22ms  cudaEventCreate
                    0.11%  350.44us         1  350.44us  350.44us  350.44us  cuDeviceTotalMem
                    0.05%  143.55us       101  1.4210us     129ns  63.151us  cuDeviceGetAttribute
                    0.02%  74.335us         1  74.335us  74.335us  74.335us  cudaLaunchKernel
                    0.01%  33.183us         1  33.183us  33.183us  33.183us  cuDeviceGetName
                    0.00%  12.900us         2  6.4500us  1.9520us  10.948us  cudaEventRecord
                    0.00%  8.8070us   

**Задание 2. Написать программу на Cи с использованием CUDA runtime API в соответствии с вариантом задания. Измерить время работы программы для различных значений параметров. Результаты занести в отчёт. Написать программу для верификации результатов.**

**Даны матрицы А и В из NxN натуральных (ненулевых) элементов (задаются случайно). Матрицы расположены в глобальной памяти.
Написать программу, выполняющую перемножение двух матриц на GPU.** 

Нужно прочесть 2N значений из глобальной памяти, выполнить 2N арифметических операций 
0. инициализация CUDA
1. выделение видеопамяти для хранения данных программы
2. копирование необходимых для работы функции данных из оперативной памяти в видеопамять
3. вызов функции CUDA
4. копирование возвращаемых данных из видеопамяти в оперативную
5. освобождение видеопамяти

In [82]:
%%writefile matrixMult.cu
#include "cuda_runtime.h"
#include <stdio.h>
#include <stdlib.h>

#define BLOCK_SIZE 16

__global__ void matrixMult(int *A, int *B, int *C, int N) 
{
    int i0 = N * (blockDim.y * blockIdx.y +  threadIdx.y);
    int j0 = blockDim.x * blockIdx.x + threadIdx.x;
    int sum = 0;

    for (int k = 0; k < N; k++)
    sum += A[i0 + k] * B[k * N + j0]; //смещение для записываемого элемента

    int ind = N * (blockDim.y * blockIdx.y + threadIdx.y) + blockDim.x * blockIdx.x + threadIdx.x;
    C[ind] = sum;
}

int main()
{
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // количество строк и столбцов матрицы
    int N = 100;
    
    size_t ABCsize = N * N * sizeof(int);
    
    int *h_A = (int *)malloc(ABCsize);
    int *h_B = (int *)malloc(ABCsize);
    int *h_C = (int *)malloc(ABCsize);
 
    //заполнение матриц
    for (int i = 0; i < N * N; ++i)
    {
        h_A[i] = rand() % (int)RAND_MAX;
    }

    for (int i = 0; i < N * N; ++i)
    {
        h_B[i] = rand()%(int)RAND_MAX;
    }
    
    int *d_A = NULL;
    cudaMalloc((void **)&d_A, Asize);
    
    int *d_B = NULL;
    cudaMalloc((void **)&d_B, Bsize);
    
    int * d_C = NULL;
    cudaMalloc((void **)&d_C, Csize);
    cudaMemcpy(d_A, h_A, ABCsize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, ABCsize, cudaMemcpyHostToDevice);
 
    dim3 threadsPerBlock = dim3(BLOCK_SIZE, BLOCK_SIZE);
    dim3 blocksPerGrid = dim3(N / BLOCK_SIZE, N /  BLOCK_SIZE);
    cudaEventRecord(start, 0);
    
    matrixMult<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);
    
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    float KernelTime;
    cudaEventElapsedTime(&KernelTime, start, stop);
    printf("KernelTime: %.2f milliseconds\n", KernelTime);
    
    cudaMemcpy(h_C, d_C, Csize, cudaMemcpyDeviceToHost);
    
    printf("MULTIPLICATE\n");
    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            int sum = 0;
                for (int k = 0; k < N; k++) 
                    sum += h_A[i * N + k] * h_B[k * N + j];
        }
    }
    printf("END\n");
    
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(h_A);
    free(h_B);
    free(h_C);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Overwriting matrixMult.cu


In [83]:
!nvcc matMult.cu -o matMult -Wno-deprecated-gpu-targets
!nvprof ./matMult

==2392== NVPROF is profiling process 2392, command: ./matMult
Arows = 112
KernelTime: 0.04 milliseconds
Test STARTED
Test PASSED
==2392== Profiling application: ./matMult
==2392== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   51.19%  21.375us         1  21.375us  21.375us  21.375us  matrixMult(int*, int*, int*, int)
                   34.87%  14.560us         2  7.2800us  6.4320us  8.1280us  [CUDA memcpy HtoD]
                   13.95%  5.8240us         1  5.8240us  5.8240us  5.8240us  [CUDA memcpy DtoH]
      API calls:   99.63%  300.91ms         2  150.46ms     955ns  300.91ms  cudaEventCreate
                    0.12%  373.97us         1  373.97us  373.97us  373.97us  cuDeviceTotalMem
                    0.07%  219.43us         3  73.142us  2.5110us  213.14us  cudaMalloc
                    0.05%  149.40us       101  1.4790us     127ns  64.348us  cuDeviceGetAttribute
                    0.05%  138.39us         